In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [3]:
API_KEY = os.getenv(key='API_KEY')

In [4]:
df = pd.read_csv('archive/steam-games.csv')
df.head(2)

,app_id,title,release_date,genres,categories,developer,publisher,original_price,discount_percentage,discounted_price,...,win_support,mac_support,linux_support,awards,overall_review,overall_review_%,overall_review_count,recent_review,recent_review_%,recent_review_count
0,730,Counter-Strike 2,"21 Aug, 2012","Action, Free to Play","Cross-Platform Multiplayer, Steam Trading Card...",Valve,Valve,NaN,NaN,Free,...,True,False,True,1,Very Positive,87.0,8062218.0,Mostly Positive,79.0,57466.0
1,570,Dota 2,"9 Jul, 2013","Action, Strategy, Free to Play","Steam Trading Cards, Steam Workshop, SteamVR C...",Valve,Valve,NaN,NaN,Free,...,True,True,True,0,Very Positive,81.0,2243112.0,Mostly Positive,72.0,23395.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42497 entries, 0 to 42496
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   app_id                42497 non-null  int64  
 1   title                 42497 non-null  object 
 2   release_date          42440 non-null  object 
 3   genres                42410 non-null  object 
 4   categories            42452 non-null  object 
 5   developer             42307 non-null  object 
 6   publisher             42286 non-null  object 
 7   original_price        4859 non-null   object 
 8   discount_percentage   4859 non-null   object 
 9   discounted_price      42257 non-null  object 
 10  dlc_available         42497 non-null  int64  
 11  age_rating            42497 non-null  int64  
 12  content_descriptor    2375 non-null   object 
 13  about_description     42359 non-null  object 
 14  win_support           42497 non-null  bool   
 15  mac_support        

- **Análise de Mercado e Tendências**
	-  *Quais são os gêneros de jogos mais populares na plataforma?*
		- identificar quais gêneros têm o maior número de jogos, maior número de vendas e melhores avaliações para orientar desenvolvedores sobre tendências de mercado.
	-  *Existe uma correlação entre a data de lançamento de um jogo e suas avaliações/reviews?*
		- Verificar se jogos lançados recentemente tendem a ter avaliações melhores ou piores que jogos mais antigos, potencialmente devido a melhorias na tecnologia ou expectativas dos jogadores.
	-  *Quais desenvolvedores e publicadores têm os jogos com melhores avaliações?*
		-  Identificar quais empresas estão produzindo jogos de alta qualidade, o que pode ser útil para parcerias, investimentos e colaboração.

In [9]:
df.isnull().sum()

app_id                      0
title                       0
release_date               57
genres                     87
categories                 45
developer                 190
publisher                 211
original_price          37638
discount_percentage     37638
discounted_price          240
dlc_available               0
age_rating                  0
content_descriptor      40122
about_description         138
win_support                 0
mac_support                 0
linux_support               0
awards                      0
overall_review           2477
overall_review_%         2477
overall_review_count     2477
recent_review           36994
recent_review_%         36994
recent_review_count     36994
dtype: int64

In [10]:
df.columns

Index(['app_id', 'title', 'release_date', 'genres', 'categories', 'developer',
       'publisher', 'original_price', 'discount_percentage',
       'discounted_price', 'dlc_available', 'age_rating', 'content_descriptor',
       'about_description', 'win_support', 'mac_support', 'linux_support',
       'awards', 'overall_review', 'overall_review_%', 'overall_review_count',
       'recent_review', 'recent_review_%', 'recent_review_count'],
      dtype='object')

In [11]:
df_1 = df[['app_id', 'title', 'release_date', 'genres', 'categories', 'developer',
       'publisher', 'awards', 'overall_review', 'overall_review_%', 
        'overall_review_count']]
df_1.head(3)

,app_id,title,release_date,genres,categories,developer,publisher,awards,overall_review,overall_review_%,overall_review_count
0,730,Counter-Strike 2,"21 Aug, 2012","Action, Free to Play","Cross-Platform Multiplayer, Steam Trading Card...",Valve,Valve,1,Very Positive,87.0,8062218.0
1,570,Dota 2,"9 Jul, 2013","Action, Strategy, Free to Play","Steam Trading Cards, Steam Workshop, SteamVR C...",Valve,Valve,0,Very Positive,81.0,2243112.0
2,2215430,Ghost of Tsushima DIRECTOR'S CUT,"16 May, 2024","Action, Adventure","Single-player, Online Co-op, Steam Achievement...",Sucker Punch Productions,PlayStation PC LLC,0,Very Positive,89.0,12294.0


In [8]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42497 entries, 0 to 42496
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   app_id                42497 non-null  int64  
 1   title                 42497 non-null  object 
 2   release_date          42440 non-null  object 
 3   genres                42410 non-null  object 
 4   categories            42452 non-null  object 
 5   developer             42307 non-null  object 
 6   publisher             42286 non-null  object 
 7   awards                42497 non-null  int64  
 8   overall_review        40020 non-null  object 
 9   overall_review_%      40020 non-null  float64
 10  overall_review_count  40020 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 3.6+ MB


In [14]:
df_1.isnull().sum()

app_id                     0
title                      0
release_date              57
genres                    87
categories                45
developer                190
publisher                211
awards                     0
overall_review          2477
overall_review_%        2477
overall_review_count    2477
dtype: int64

In [16]:
df_1 = df_1.dropna()

In [17]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39853 entries, 0 to 42248
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   app_id                39853 non-null  int64  
 1   title                 39853 non-null  object 
 2   release_date          39853 non-null  object 
 3   genres                39853 non-null  object 
 4   categories            39853 non-null  object 
 5   developer             39853 non-null  object 
 6   publisher             39853 non-null  object 
 7   awards                39853 non-null  int64  
 8   overall_review        39853 non-null  object 
 9   overall_review_%      39853 non-null  float64
 10  overall_review_count  39853 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 3.6+ MB


In [18]:
df_1[['overall_review', 'overall_review_%', 
        'overall_review_count']]

,overall_review,overall_review_%,overall_review_count
0,Very Positive,87.0,8062218.0
1,Very Positive,81.0,2243112.0
2,Very Positive,89.0,12294.0
3,Very Positive,93.0,605191.0
4,Very Positive,80.0,594713.0
...,...,...,...
40950,Very Negative,14.0,57.0
40968,Very Negative,14.0,54.0
41427,Overwhelmingly Negative,17.0,3299.0
42247,Very Negative,18.0,127.0


In [5]:
def _chat(df_path):
    openai_api_key = API_KEY
    df = df_path
    llm = ChatOpenAI(model = "gpt-3.5-turbo", openai_api_key=openai_api_key)
    agent_prefix = "O df importado é\n"

    agent = create_pandas_dataframe_agent(
        llm,
        df,
        prefix=agent_prefix,
        verbose  = True,
        agent_type = AgentType.OPENAI_FUNCTIONS,
    )

    while True:
        prompt = input("Digite o prompt ou 'sair\n'")
        if prompt.lower() == 'sair':
            print("finalizando...")
            break
        response = agent.invoke(prompt)

In [7]:
_chat(df)

Digite o prompt ou 'sair
' DESCRIBE




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.describe()'}`


             app_id  dlc_available    age_rating        awards  \
count  4.249700e+04   42497.000000  42497.000000  42497.000000   
mean   1.245812e+06       0.722004      0.055886      0.309528   
std    7.553906e+05       4.673810      0.229705      1.264100   
min    1.000000e+01       0.000000      0.000000      0.000000   
25%    5.892200e+05       0.000000      0.000000      0.000000   
50%    1.144860e+06       0.000000      0.000000      0.000000   
75%    1.822640e+06       0.000000      0.000000      0.000000   
max    3.008050e+06     200.000000      1.000000     41.000000   

       overall_review_%  overall_review_count  recent_review_%  \
count      40020.000000          4.002000e+04      5503.000000   
mean          77.162569          2.500621e+03        82.303107   
std           17.672036          4.887980e+04        15.110260   
min            0.000000          1.0

Digite o prompt ou 'sair
' sair


finalizando...
